# 第8章 计算性能

本章将重点介绍影响计算性能的重要因子：命令式编程、符号式编程、异步计算、自动并行计算和多GPU计算。

## 8.1 命令式和符号式混合编程

In [2]:
def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

fancy_func(1, 2, 3, 4)

10

与命令式编程不同，符号式编程通常在计算流程完全定义好后才被执行。通常，符号式编程的程序需要下面3个步骤：
- 1.定义计算流程；
- 2.把计算流程编译成可执行的程序；
- 3.给定输入，调用编译好的程序执行。

In [1]:
def add_str():
    return '''
def add(a, b):
    return a + b
'''

def fancy_func_str():
    return '''
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
'''

def evoke_str():
    return add_str() + fancy_func_str() + '''
print(fancy_func(1, 2, 3, 4))
'''
prog = evoke_str()
print(prog)
y = compile(prog, '', 'exec')
exec(y)


def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

print(fancy_func(1, 2, 3, 4))

10


可以看到：
- 命令式编程更方便；
- 符号式编程更高效并更容易移植。

### 8.1.1 混合式编程取两者之长

开发者们认为，用户应该用纯命令式编程进行开发和调试；当需要产品级别的计算性能和部署时，用户可以将大部分命令式程序转换成符号式程序来运行。

在混合式编程中，我们可以通过使用HybridBlock类或者HybridSequential类构建模型。默认情况下，它们和Block类或者Sequential类一样依据命令式编程的方式执行。当我们调用hybridize函数后，Gluon会转换成依据符号式名称的方式执行。

### 8.1.2 使用HybridSequential类构造模型

In [2]:
from mxnet import nd, sym
from mxnet.gluon import nn
import time

def get_net():
    net = nn.HybridSequential() # 这里创建HybridSequential实例
    net.add(nn.Dense(256, activation='relu'),
            nn.Dense(128, activation='relu'),
            nn.Dense(2))
    net.initialize()
    return net

x = nd.random_normal(shape=(1,512))
net = get_net()
net(x)


[[0.08811311 0.06387279]]
<NDArray 1x2 @cpu(0)>

我们可以通过调用hybridize函数来编译和优化HybridSequential实例中串联的层的计算。模型的计算结果不变。

In [3]:
net.hybridize()
net(x)


[[0.08811311 0.06387279]]
<NDArray 1x2 @cpu(0)>

需要注意的是，只有继承HybridBlock类的层才会被优化计算。

#### 计算性能

In [4]:
def benchmark(net, x):
    start = time.time()
    for i in range(1000):
        _ = net(x)
    nd.waitall() # 等待所有计算完成方便计时
    return time.time() - start

net = get_net()
print('before hybridizing: %.4f sec' % (benchmark(net, x)))
net.hybridize()
print('after hybridizing: % .4f sec' % (benchmark(net, x)))

before hybridizing: 0.2334 sec
after hybridizing:  0.1326 sec


#### 获取符号式程序

在模型net根据输入计算出模型输出后，我们就可以通过export函数将符号式程序和模型参数保存到硬盘。

In [5]:
net.export('my_mlp')

此时生成的.json和.params文件分别为符号式程序和模型参数。它们可以被Python或MXNet支持的其他前端语言读取，如C++、R、Scala、Perl和其他语言。这样，我们就可以很方便地使用其他前端语言或在其他设备上部署训练好的模型。同时，由于部署时使用的是符号式程序，计算性能往往比命令式程序的性能更好。

在MXNet中，符号式程序指的是基于`Symbol`类型的程序。对于调用过`hybridize`函数后的模型，我们还可以给它输入一个`Symbol`类型的变量，`net(x)`会返回`Symbol`类型的结果。

In [6]:
x = sym.var('data')
net(x)

<Symbol dense5_fwd>

### 8.1.3 使用HybridBlock类构造模型

和`Sequential`类与`Block`类之间的关系一样，`HybridSequential`类是`HybridBlock`类的子类。与`Block`实例需要实现`forward`函数不太一样的是，对于`HybridBlock`实例，我们需要实现`hybrid_forward`函数。

前面我们展示了调用`hybridize`函数后的模型可以获得更好的计算性能和可移植性。此外，调用`hybridize`函数后的模型会影响灵活性。

In [7]:
class HybridNet(nn.HybridBlock):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.hidden = nn.Dense(10)
        self.output = nn.Dense(2)
        
    def hybrid_forward(self, F, x):
        print('F: ', F)
        print('x: ', x)
        x = F.relu(self.hidden(x))
        print('hidden: ', x)
        return self.output(x)

在继承`HybridBlock`类时，我们需要在`hybrid_forward`函数中添加额外的输入`F`。我们知道，MXNet既有基于命令式编程的`NDArray`类，又有基于符号式编程的`Symbol`类。由于这两个类的函数基本一致，MXNet会根据输入来决定`F`使用`NDArray`或`Symbol`。

In [8]:
net = HybridNet()
net.initialize()
x = nd.random_normal(shape=(1, 4))
net(x)

F:  <module 'mxnet.ndarray' from 'd:\\anaconda3\\install\\envs\\gluon\\lib\\site-packages\\mxnet\\ndarray\\__init__.py'>
x:  
[[ 0.02184284 -0.31464806 -0.33364916 -0.6471778 ]]
<NDArray 1x4 @cpu(0)>
hidden:  
[[0.         0.02384557 0.         0.01206701 0.         0.02765122
  0.         0.03072213 0.02471942 0.        ]]
<NDArray 1x10 @cpu(0)>



[[-0.00021427 -0.00183663]]
<NDArray 1x2 @cpu(0)>

In [9]:
net(x)

F:  <module 'mxnet.ndarray' from 'd:\\anaconda3\\install\\envs\\gluon\\lib\\site-packages\\mxnet\\ndarray\\__init__.py'>
x:  
[[ 0.02184284 -0.31464806 -0.33364916 -0.6471778 ]]
<NDArray 1x4 @cpu(0)>
hidden:  
[[0.         0.02384557 0.         0.01206701 0.         0.02765122
  0.         0.03072213 0.02471942 0.        ]]
<NDArray 1x10 @cpu(0)>



[[-0.00021427 -0.00183663]]
<NDArray 1x2 @cpu(0)>

In [10]:
net.hybridize()
net(x)

F:  <module 'mxnet.symbol' from 'd:\\anaconda3\\install\\envs\\gluon\\lib\\site-packages\\mxnet\\symbol\\__init__.py'>
x:  <Symbol data>
hidden:  <Symbol hybridnet0_relu0>



[[-0.00021427 -0.00183663]]
<NDArray 1x2 @cpu(0)>

In [13]:
x = nd.random_normal(shape=(1, 4))
net(x)


[[ 0.00118421 -0.00075799]]
<NDArray 1x2 @cpu(0)>

对于少数像`asnumpy`这样的`Symbol`所不支持的函数，以及像`a += b`和`a[:] = a + b`（需改写为`a = a + b`）这样的原地（in-place）操作，我们无法在`hybrid_forward`函数中使用并在调用`hybridize`函数后进行前向计算。

### 小结

- 命令式编程和符号式编程各有优劣。MXNet通过混合式编程取二者之长。
- 通过`HybridSequential`类和`HybridBlock`类构建的模型可以调用`hybridize`函数将命令式程序转成符号式程序。